In [3]:
import pandas as pd
import requests
import re
import json
import csv
import glob
from pandas.io.json import json_normalize

In [4]:
# concatenate 50 csvs of "People from XX town" pages (one for each state)
path =r'/hi/im/a/fake/path/name' 
allFiles = glob.glob(path + "/*.csv")

list_ = []

for file_ in allFiles:
    df = pd.read_csv(file_,index_col=None, header=0)
    list_.append(df)

df = pd.concat(list_, axis = 0, ignore_index = True)

In [5]:
df.head()
df.shape

(2185, 3)

In [9]:
df

,pageid,ns,title
0,53129812,14,"Category:People from Belleair, Florida"
1,53015201,14,"Category:People from Century, Florida"
2,53045468,14,"Category:People from Greenville, Florida"
3,53028329,14,"Category:People from Groveland, Florida"
4,53118985,14,"Category:People from Gulf Stream, Florida"
5,53117670,14,"Category:People from Highland Beach, Florida"
6,53089455,14,"Category:People from Hilliard, Florida"
7,53155424,14,"Category:People from Hillsboro Beach, Florida"
8,52986036,14,"Category:People from Indialantic, Florida"
9,53134395,14,"Category:People from Interlachen, Florida"


In [7]:
# clean up extraneous "municipalities" pages
df = df[~df.title.str.contains("municipalities")]

In [8]:
df.shape

(2184, 3)

In [10]:
# double check to make sure all remaining rows are "People from XX city" pages
df[df.title.str.contains('People from')].count()

pageid    2184
ns        2184
title     2184
dtype: int64

In [11]:
# save to CSV
df.to_csv("category_people_from_all_towns.csv", index=False)

In [12]:
# read in the new CSV
towns_df = pd.read_csv('category_people_from_all_towns.csv')

In [13]:
towns_df.head()

,pageid,ns,title
0,53129812,14,"Category:People from Belleair, Florida"
1,53015201,14,"Category:People from Century, Florida"
2,53045468,14,"Category:People from Greenville, Florida"
3,53028329,14,"Category:People from Groveland, Florida"
4,53118985,14,"Category:People from Gulf Stream, Florida"


In [17]:
# loop through each row and make an API call for each sub-page
# then save to a json file

categories = df['title']

for category in categories:
    url = "http://en.wikipedia.org/w/api.php?action=query&list=categorymembers&cmtitle=" + category + "&cmtype=page&cmlimit=500&format=json"
    response = requests.get(url)
    
    with open(category+'.json', 'w') as outf:
        outf.write(response.text)

In [18]:
path =r'/blah/blah/blah' 
files = glob.glob(path + "/*.json")

list = []

for file in files:
    with open(file) as f:

        data = json.load(f)
        data2 = json_normalize(data['query'])
        data3 = json_normalize(data=data['query'], record_path='categorymembers', )
        data3['filename'] = file
        
        list.append(data3)

df = pd.concat(list, axis = 0, ignore_index = True, sort=False)

In [112]:
pd.set_option('display.max_colwidth', -1)

In [20]:
df['name'] = df['title']

In [21]:
df['city'] = df['filename'].str.extract(r'from (.*).json')

In [22]:
df = df.drop(['title', 'ns', 'filename'], axis=1)

In [23]:
df

,pageid,name,city
0,5848474.0,Doug Barfield,"Grove Hill, Alabama"
1,19281838.0,Zachariah Bettis,"Grove Hill, Alabama"
2,13364129.0,John W. Cranford,"Grove Hill, Alabama"
3,13567065.0,Grant Gillis,"Grove Hill, Alabama"
4,44230987.0,Winky Hicks,"Grove Hill, Alabama"
5,40654882.0,Marc Keahey,"Grove Hill, Alabama"
6,1664465.0,F. David Mathews,"Grove Hill, Alabama"
7,13531563.0,Cliff Nobles,"Grove Hill, Alabama"
8,14579208.0,I. T. Quinn,"Grove Hill, Alabama"
9,3351993.0,Ebenezer Allen,"Poultney (town), Vermont"


In [24]:
df.pageid=df.pageid.astype(int)

In [25]:
df.head()

,pageid,name,city
0,5848474,Doug Barfield,"Grove Hill, Alabama"
1,19281838,Zachariah Bettis,"Grove Hill, Alabama"
2,13364129,John W. Cranford,"Grove Hill, Alabama"
3,13567065,Grant Gillis,"Grove Hill, Alabama"
4,44230987,Winky Hicks,"Grove Hill, Alabama"


In [26]:
# save to CSV

df.to_csv("category_people_from_towns_allpeople.csv", index=False)